# config

In [ ]:
from getpass import getpass
import requests, json

token = getpass("Enter GitHub token (hidden): ")

r = requests.get("https://api.github.com/user", headers={"Authorization": f"token {token}"})
print(r.status_code)
print(r.json().get("login"))

Enter GitHub token (hidden): ··········
200
Vu-Quoc-Tuan


In [ ]:
username = "Vu-Quoc-Tuan"
repo_name = "BTL-NLP-2526I_INT3406_3"

# Sử dụng biến token để xác thực trực tiếp trong URL
!git clone https://{token}@github.com/{username}/{repo_name}

Cloning into 'BTL-NLP-2526I_INT3406_3'...
remote: Enumerating objects: 243, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 243 (delta 35), reused 46 (delta 21), pack-reused 177 (from 1)
Receiving objects: 100% (243/243), 136.89 MiB | 26.05 MiB/s, done.
Resolving deltas: 100% (116/116), done.
Updating files: 100% (49/49), done.


In [ ]:
# 1) vào thư mục repo (thay path nếu khác)
%cd /content/BTL-NLP-2526I_INT3406_3

# 2) kiểm tra hiện tại đang ở đâu và liệt kê file
!pwd
!ls -la
!ls -la vlsp-mt
!test -f vlsp-mt/requirements.txt && echo "requirements found" || echo "requirements NOT found"

# 3) show python interpreter đang dùng
import sys
print("Python executable:", sys.executable)

# 4) dùng chính interpreter đó để cài (an toàn hơn dùng !pip trực tiếp)
!{sys.executable} -m pip install -r vlsp-mt/requirements.txt

/content/BTL-NLP-2526I_INT3406_3
/content/BTL-NLP-2526I_INT3406_3
total 28
drwxr-xr-x 5 root root 4096 Dec 22 14:00 .
drwxr-xr-x 1 root root 4096 Dec 22 14:00 ..
drwxr-xr-x 8 root root 4096 Dec 22 14:00 .git
-rw-r--r-- 1 root root   60 Dec 22 14:00 .gitignore
-rw-r--r-- 1 root root   25 Dec 22 14:00 README.md
drwxr-xr-x 4 root root 4096 Dec 22 14:00 transformer_base
drwxr-xr-x 4 root root 4096 Dec 22 14:00 vlsp-mt
total 80
drwxr-xr-x 4 root root  4096 Dec 22 14:00 .
drwxr-xr-x 5 root root  4096 Dec 22 14:00 ..
drwxr-xr-x 6 root root  4096 Dec 22 14:00 data
-rw-r--r-- 1 root root 39689 Dec 22 14:00 REPORT.md
-rw-r--r-- 1 root root   485 Dec 22 14:00 requirements.txt
-rw-r--r-- 1 root root 11658 Dec 22 14:00 RUN_GUIDE.md
-rw-r--r-- 1 root root  4621 Dec 22 14:00 run.md
drwxr-xr-x 3 root root  4096 Dec 22 14:00 scripts
requirements found
Python executable: /usr/bin/python3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Run script

## Preprocessing

In [ ]:
%cd vlsp-mt

/content/BTL-NLP-2526I_INT3406_3/vlsp-mt


## Train

In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.7 MB/s eta 0:00:00


In [6]:
!python scripts/train_qwen_lora.py \
  --model_name Qwen/Qwen2.5-3B-Instruct \
  --direction vi2en \
  --src data/dedup/train.vi \
  --tgt data/dedup/train.en \
  --val_src data/clean/dev.vi \
  --val_tgt data/clean/dev.en \
  --run_id vi2en_v1 \
  --epochs 2 \
  --batch_size 32 \
  --grad_accum 4 \
  --lr 1.5e-4 \
  --lora_r 32 \
  --lora_alpha 64 \
  --neftune_alpha 3.0 \
  --eval_bleu \
  --bleu_sample_size 100 \
  --eval_steps 2000 \
  --early_stopping_patience 3 \
  --max_len 256 \
  --no_grad_checkpoint  \
  --medical_vocab data/medical_vocab.txt \
  --init_new_embeddings_avg

2025-12-22 14:01:07.541210: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-22 14:01:07.559030: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766412067.579988     520 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766412067.586519     520 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766412067.603185     520 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [7]:
# 1. Tạo thư mục đích trước
!mkdir -p data/rl_subset

# 2. Chạy lệnh cắt file
!head -n 20000 data/dedup/train.en > data/rl_subset/en.txt
!head -n 20000 data/dedup/train.vi > data/rl_subset/vi.txt

!ls -lh data/rl_subset/

total 6.3M
-rw-r--r-- 1 root root 2.8M Dec 22 19:59 en.txt
-rw-r--r-- 1 root root 3.6M Dec 22 19:59 vi.txt


## Eval

In [23]:
!python scripts/generate.py \
    --model_name Qwen/Qwen2.5-3B-Instruct \
    --adapter_path runs/vi2en_v1/lora_vi2en_sft \
    --direction vi2en \
    --input data/clean/test.vi \
    --output outputs/test_vi2en_sft.hyp.en \
    --batch_size 24 \
    --num_beams 4 \
    --repetition_penalty 1.1

2025-12-22 20:14:20.578250: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-22 20:14:20.596673: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766434460.619264   91978 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766434460.626226   91978 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766434460.643477   91978 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [24]:
!python scripts/eval_bleu.py \
    --hyp outputs/test_vi2en_sft.hyp.en \
    --ref data/clean/test.en \
    --src data/clean/test.vi \
    --show_worst 10

Loading hypothesis: outputs/test_vi2en_sft.hyp.en
Loading reference: data/clean/test.en
Loading source: data/clean/test.vi

Evaluating 1000 sentence pairs...

EVALUATION RESULTS
  BLEU:    43.72
  chrF++:  63.87
  chrF:    65.96
  TER:     49.06 (lower is better)
  METEOR:  69.82

10 WORST TRANSLATIONS (by BLEU):
--------------------------------------------------

[1] Line 564 | BLEU: 0.0 | chrF: 53.7
  SRC: Phẫu thuật nạo sàng qua nội soi
  HYP: Endoscopic ethmoidectomy
  REF: Ethmoidectomy via endoscopic technique

[2] Line 596 | BLEU: 0.0 | chrF: 15.7
  SRC: Vai trò phẫu thuật glenn trong điều trị bệnh tim bẩm sinh tím
  HYP: The role of glenn operation in the treatment of cyanotic congenital heart disease
  REF: Minimally invasive aortic valve surgery

[3] Line 860 | BLEU: 2.0 | chrF: 0.8
  SRC: Kết quả điều trị viêm phổi nặng ở trẻ từ 1tháng đến 5 tuổi tại Quảng Ninh
  HYP: Results of treatment of severe diarrhea in children from 1 month to 5 years old in Quang Ninh
  REF: RESULT 

# Public test

In [28]:
!python scripts/generate.py \
    --model_name Qwen/Qwen2.5-3B-Instruct \
    --adapter_path runs/vi2en_v1/lora_vi2en_sft \
    --direction vi2en \
    --input data/raw/test_unseen_v3.vi.txt \
    --output outputs/test_unseen_v3.hyp.en \
    --batch_size 24 \
    --num_beams 4 \
    --repetition_penalty 1.1

2025-12-22 21:18:10.593362: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-22 21:18:10.612170: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766438290.634202  108253 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766438290.640820  108253 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766438290.657697  108253 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [29]:
!python scripts/eval_bleu.py \
    --hyp outputs/test_unseen_v3.hyp.en \
    --ref data/raw/test_unseen_v3.en.txt \
    --src data/raw/test_unseen_v3.vi.txt \
    --show_worst 10

Loading hypothesis: outputs/test_unseen_v3.hyp.en
Loading reference: data/raw/test_unseen_v3.en.txt
Loading source: data/raw/test_unseen_v3.vi.txt

Evaluating 1152 sentence pairs...

EVALUATION RESULTS
  BLEU:    37.84
  chrF++:  60.23
  chrF:    62.69
  TER:     55.61 (lower is better)
  METEOR:  66.32

10 WORST TRANSLATIONS (by BLEU):
--------------------------------------------------

[1] Line 247 | BLEU: 0.0 | chrF: 0.5
  SRC: Nhận xét một số đặc điểm lâm sàng, cận lâm sàng bệnh nhân thay van hai lá cơ học có bảo tồn lá sau tại bệnh viện Trung ương Huế
  HYP: Remarks on some clinical and subclinical characteristics of patients with mechanical mitral valve replacement with posterior leaflet preservation at Hue Central Hospital
  REF: ASSESSMENT THE CLINICAL AND SUBCLINICAL IN PATIENTS WHO UNDERWENT MICHANICAL MITRL VALVE REPLACEMENT WITH POSTERIOR LEAFLET PRESERVATION AT HUE CENTER HOSPITAL

[2] Line 848 | BLEU: 0.2 | chrF: 17.8
  SRC: ﻿HIỆU QUẢ ĐIỀU TRỊ BỆNH XƠ CỨNG BÌ HỆ THỐNG TÓM

In [26]:
!python scripts/generate.py \
    --model_name Qwen/Qwen2.5-3B-Instruct \
    --adapter_path runs/vi2en_v1/lora_vi2en_sft \
    --direction vi2en \
    --input data/raw/public_test.vi.txt \
    --output outputs/public_test_vi2en_sft.hyp.en \
    --batch_size 24 \
    --num_beams 4 \
    --repetition_penalty 1.1

2025-12-22 20:45:52.126983: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-22 20:45:52.144957: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766436352.166204  100113 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766436352.172755  100113 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766436352.189361  100113 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [27]:
!python scripts/eval_bleu.py \
    --hyp outputs/public_test_vi2en_sft.hyp.en \
    --ref data/raw/public_test.en.txt \
    --src data/raw/public_test.vi.txt \
    --show_worst 10

Loading hypothesis: outputs/public_test_vi2en_sft.hyp.en
Loading reference: data/raw/public_test.en.txt
Loading source: data/raw/public_test.vi.txt

Evaluating 3000 sentence pairs...

EVALUATION RESULTS
  BLEU:    42.54
  chrF++:  63.30
  chrF:    65.54
  TER:     51.91 (lower is better)
  METEOR:  69.36

10 WORST TRANSLATIONS (by BLEU):
--------------------------------------------------

[1] Line 295 | BLEU: 0.0 | chrF: 66.0
  SRC: Rối loạn lipid máu do thuốc
  HYP: Drug-induced dyslipidemia
  REF: Drug - Induced Dyslipidemia

[2] Line 457 | BLEU: 0.0 | chrF: 0.5
  SRC: Nhận xét một số đặc điểm lâm sàng, cận lâm sàng bệnh nhân thay van hai lá cơ học có bảo tồn lá sau tại bệnh viện Trung ương Huế
  HYP: Remarks on some clinical and subclinical characteristics of patients with mechanical mitral valve replacement with posterior leaflet preservation at Hue Central Hospital
  REF: ASSESSMENT THE CLINICAL AND SUBCLINICAL IN PATIENTS WHO UNDERWENT MICHANICAL MITRL VALVE REPLACEMENT WITH POST